## Install the library

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.

## Import the Libraries

In [2]:
import os
import ee
import geemap
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium

## Authenticate and Initialise with your account associated with Google Earth Engine

In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-WnGhaOL-KNY6XVszyp0YyH7LpbjygB6qoaJQpVYVyM&tc=pPDVWMI6gMhGz7XF9fc4GRR3DJ-LRs-bfik9d1XmkZs&cc=dRQVAPsu_MTui8Pqt3yKPEU6iWHHgaw-rUyPSc5-LrU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh7Fvp7C-LoaMdY4Ll-50-rhJ9xzpvTy9zqhYZkjdfmy5vdMcL4gXLI

Successfully saved authorization token.


## Data Range - 1 Jan 2013 to 31 Dec 2022


In [4]:
# Initialize date range
start_date = ee.Date('2013-01-01')
end_date = ee.Date('2022-12-31')

# Finding water quality parameters

## Sentinel-2A

We have to take NDWI (Normalized Difference Water Index), How ? 

> NDWI = (B3 - B8) / (B3 + B8)

The green region (B3, centered at 560 nm).

The near-infrared region (B8, centered at 842 nm)





### Raster to Vector Conversion

In [7]:
Map = geemap.Map()

geometry = ee.Geometry.Polygon([[77.39023917078465, 23.27477380472318],
          [77.39034645914525, 23.274596401498602],
          [77.39034645914525, 23.273876930443446],
          [77.39060395121068, 23.273847363056678],
          [77.3905717647025, 23.27359111209644],
          [77.3909150874564, 23.273729093443976],
          [77.39105456232518, 23.273847363056678],
          [77.39120476603001, 23.273876930443446],
          [77.39134424089879, 23.27427116163988],
          [77.39111893534154, 23.275808652157437],
          [77.39099018930882, 23.27569038428615],
          [77.39077561258763, 23.27569038428615],
          [77.3904108321616, 23.275394714148526]])


In [9]:
image = ee.ImageCollection("COPERNICUS/S2_SR") \
            .filterBounds(geometry) \
            .first()
# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B8'])
# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1= ndwiMasked.toInt()
vectors = ndwiMasked1.reduceToVectors(scale = 30.0, geometryType = 'polygon', eightConnected = False, maxPixels = 10000000, bestEffort=True)


In addition, you can also filter image collections based on bands, using methods such as 

*  ee.Filter.eq() (equals),
*  ee.Filter.neq() (not equals),
*  ee.Filter.lt() (less than),
*  ee.Filter.lte() (less than or equal to)
*  ee.Filter.gt() (greater than),
*  ee.Filter.gte() (greater than or equal to),
*  ee.Filter.range() (a range of values),
*  ee.Filter.inList() (a list of values).

## What kind of things can we used instead of **cloud_pixel_percentage**
1.   **CLOUD_COVER:** The overall percentage of cloudy pixels in the image.
2.   **CLOUDY_PIXEL_PERCENTAGE:** The percentage of cloudy pixels in the image, excluding pixels that are designated as "shadow" or "snow/ice".
3.  **SUN_ELEVATION:** The solar elevation angle in degrees.
4.   **SUN_AZIMUTH_ANGLE:** The solar azimuth angle in degrees.
5.   **IMAGE_QUALITY:** A quality assessment value for the image, ranging from 0 (lowest quality) to 100 (highest quality).
6. **SENSING_TIME:** The time at which the image was acquired, in milliseconds since the Unix epoch.







## Visulaization of NDWI Masked

In [10]:
Map.centerObject(geometry, 12)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 3000}, 'RGB')
Map.addLayer(ndwiMasked1, {'palette': '00FFFF, 0000FF'}, 'NDWI Masked')
Map.addLayer(vectors, {}, 'Vectors')
Map.add_legend(title='NDWI Values', legend_dict={1: 'Water'})
Map

The provided color (00FFFF, 0000FF) is invalid. Using the default black color.
'#00FFFF, 0000FF' is not in web format. Need 3 or 6 hex digit.


Map(center=[23.274674871781986, 77.39080523276021], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Lendiya Lake, Bhopal
geometry = ee.Geometry.Polygon([
          [77.39023917078465, 23.27477380472318],
          [77.39034645914525, 23.274596401498602],
          [77.39034645914525, 23.273876930443446],
          [77.39060395121068, 23.273847363056678],
          [77.3905717647025, 23.27359111209644],
          [77.3909150874564, 23.273729093443976],
          [77.39105456232518, 23.273847363056678],
          [77.39120476603001, 23.273876930443446],
          [77.39134424089879, 23.27427116163988],
          [77.39111893534154, 23.275808652157437],
          [77.39099018930882, 23.27569038428615],
          [77.39077561258763, 23.27569038428615],
          [77.3904108321616, 23.275394714148526]])

Map.addLayer(geometry)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
               .filterDate(start_date,end_date) \
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
               .median()

mndwi = sentinel.normalizedDifference(['B3','B11']).rename('mndwi')
mndwitr = mndwi.gt(0)
ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')
ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')
ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')
ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')
dissolvedoxygen  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')


NameError: ignored

### Visualisation

In [ ]:
# Create a Map object
Map = geemap.Map()

# Set the center of the map and the zoom level
Map.setCenter(77.39023917078465, 23.27477380472318, 14)

# Add the geometry of Lendiya Lake
Map.addLayer(geometry, {'color': 'red'}, 'Lendiya Lake')

# Load the Sentinel-2 image collection
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(geometry) \
               .filterDate('2022-01-01','2022-12-31') \
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
               .median()

# Compute the MNDWI
mndwi = sentinel.normalizedDifference(['B3','B11']).rename('MNDWI')

# Threshold the MNDWI to create a binary water mask
water_mask = mndwi.gt(0)

# Compute the NDSI
ndsi = sentinel.normalizedDifference(['B11','B12']).rename('NDSI')

# Compute the NDTI
ndti = sentinel.normalizedDifference(['B4','B3']).rename('NDTI')

# Add the MNDWI, NDSI, and NDTI to the map
Map.addLayer(mndwi, {'min':-1, 'max':1, 'palette':['white','blue']}, 'MNDWI')
Map.addLayer(ndsi, {'min':-1, 'max':1, 'palette':['black','cyan']}, 'NDSI')
Map.addLayer(ndti, {'min':-1, 'max':1, 'palette':['pink','brown']}, 'NDTI')

# Display the map
Map


Map(center=[23.27477380472318, 77.39023917078465], controls=(WidgetControl(options=['position', 'transparent_b…

### Landsat-8

In [ ]:
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate(start_date,end_date) \
.filterBounds(vectors).median()

temp  = col.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')

### Sentinel-3

### Test Data

In [ ]:
starting = '2021-01-01'
ending = '2021-08-31'

data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(starting, ending).filterBounds(vectors)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)
dm_2021_Jan_August_test = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter_2021_Jan_August_test= rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')


latlon = ee.Image.pixelLonLat().addBands(dm_2021_Jan_August_test)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_Jan_August_test = np.array((ee.Array(latlon.get("dom")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(suspended_matter_2021_Jan_August_test)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_Jan_August_test= np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))

### Training Data

In [ ]:
start1 = '2021-01-01'
end1 = '2021-06-30'

In [ ]:
data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(start1, end1).filterBounds(vectors)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)
dm_2021_Jan_June = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter__2021_Jan_June = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(dm_2021_Jan_June)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_Jan_June = np.array((ee.Array(latlon.get("dom")).getInfo()))

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(suspended_matter__2021_Jan_June)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_Jan_June= np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))

In [ ]:
start2 = '2021-07-31'
end2 = '2021-10-31'

data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(start2, end2).filterBounds(vectors)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)
dm_2021_July_Oct = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter__2021_July_Oct = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')

latlon = ee.Image.pixelLonLat().addBands(dm_2021_July_Oct)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_July_Oct = np.array((ee.Array(latlon.get("dom")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(suspended_matter__2021_July_Oct)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_July_Oct= np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))



start3 = '2021-11-30'
end3 = '2021-12-31'

data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(start3, end3).filterBounds(vectors)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)
dm_2021_Nov_Dec = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter__2021_Nov_Dec = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')

latlon = ee.Image.pixelLonLat().addBands(dm_2021_Nov_Dec)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_Nov_Dec = np.array((ee.Array(latlon.get("dom")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(suspended_matter__2021_Nov_Dec)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_Nov_Dec = np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))

### Temperature Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(temp)


latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);

data_lst = np.array((ee.Array(latlon.get("temp")).getInfo()))
# geemap.ee_to_numpy(dataset, region=geometry)

### Turbidity Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ndti)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);
# get data into three different arrays
data_ndti = np.array((ee.Array(latlon.get("ndti")).getInfo()))

### Salinity Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ndsi)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);
# get data into three different arrays
data_ndsi = np.array((ee.Array(latlon.get("ndsi")).getInfo()))

### Chlorophyll Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ndci)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);
# get data into three different arrays
data_ndci = np.array((ee.Array(latlon.get("ndci")).getInfo()))

### Dissolved Oxygen Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_do = np.array((ee.Array(latlon.get("dissolvedoxygen")).getInfo()))

### pH Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ph)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);
# get data into three different arrays
data_ph = np.array((ee.Array(latlon.get("ph")).getInfo()))

## Concating dissolved organic matter and suspended matter values

In [ ]:
dm = [pd.DataFrame(data_dom_2021_Jan_June, columns = ['Dissolved Organic Matter']),\
      pd.DataFrame(data_dom_2021_July_Oct, columns = ['Dissolved Organic Matter']),\
      pd.DataFrame(data_dom_2021_Nov_Dec, columns = ['Dissolved Organic Matter'])]

sm = [pd.DataFrame(data_sm_2021_Jan_June, columns = ['Suspended Matter']),\
      pd.DataFrame(data_sm_2021_July_Oct, columns = ['Suspended Matter']),\
      pd.DataFrame(data_sm_2021_Nov_Dec, columns = ['Suspended Matter'])]
data_dom = pd.concat(dm, sort=False, ignore_index=True)
data_sm = pd.concat(sm, sort=False, ignore_index=True)

## Creating Dataframe

In [ ]:
df = pd.concat([pd.DataFrame(data_do, columns = ['Dissolved Oxygen']),\
           pd.DataFrame(data_ndsi, columns = ['Salinity']),\
           pd.DataFrame(data_lst, columns = ['Temperature']),\
           pd.DataFrame(data_ph, columns = ['pH']),\
           pd.DataFrame(data_ndti, columns = ['Turbidity']),\
           pd.DataFrame(data_dom, columns = ['Dissolved Organic Matter']),\
           pd.DataFrame(data_sm, columns = ['Suspended Matter']),\
           pd.DataFrame(data_ndci, columns = ['Chlorophyll'])], axis=1, sort=False)

NameError: ignored

In [ ]:
df

NameError: ignored

## Converting to CSV files

In [ ]:
df.to_csv('Data_2021_Tappar_Lake',index=False)

## Test Data

In [ ]:
df2 = pd.concat([pd.DataFrame(data_do, columns = ['Dissolved Oxygen']),\
           pd.DataFrame(data_ndsi, columns = ['Salinity']),\
           pd.DataFrame(data_lst, columns = ['Temperature']),\
           pd.DataFrame(data_ph, columns = ['pH']),\
           pd.DataFrame(data_ndti, columns = ['Turbidity']),\
           pd.DataFrame(data_dom_2021_Jan_August_test, columns = ['Dissolved Organic Matter']),\
           pd.DataFrame(data_sm_2021_Jan_August_test, columns = ['Suspended Matter']),\
           pd.DataFrame(data_ndci, columns = ['Chlorophyll'])], axis=1, sort=False)
df2.head()
df2.to_csv('Test_Data_2021_Jan_August_Kankaria_Lake',index=False)

In [ ]:
# Define the image collection and filter it
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                .filterBounds(geometry) \
                .filterDate('2013-01-01','2022-12-31') \
                .filterMetadata('CLOUD_COVER', 'less_than', 5)

# Set visualization parameters
vizParams = {'bands': ['B5', 'B6', 'B4'], 'min': 0, 'max': 4000, 'gamma': [1, 0.9, 1.1]}
vizParams2 = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}

# Get a list of images in the collection
imageList = collection.toList(collection.size())
print("List of all the images", imageList.getInfo())

# Create an empty feature collection to store the data for all images
fc = ee.FeatureCollection([])
print("it's an empty list for feature collection", fc.getInfo())

# Iterate over each image in the collection
for i in range(imageList.size().getInfo()):
  
    # Get the image from the list and cast it to ee.Image
    image = ee.Image(imageList.get(i))
    
    # Select the thermal band
    thermal = image.select('B10').multiply(0.1)

    # Compute the emissivity
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    min = ee.Number(ndvi.reduceRegion({
      'reducer': ee.Reducer.min(),
      'geometry': geometry,
      'scale': 30,
      'maxPixels': 1e9
    }).values().get(0))
    
    max = ee.Number(ndvi.reduceRegion({
      'reducer': ee.Reducer.max(),
      'geometry': geometry,
      'scale': 30,
      'maxPixels': 1e9
    }).values().get(0))
    
    fv = ndvi.subtract(min).divide(max.subtract(min)).pow(ee.Number(2)).rename('FV')
    
    a = ee.Number(0.004)
    b = ee.Number(0.986)
    
    EM = fv.multiply(a).add(b).rename('EMM')

    # Compute the LST
    lstExpression = '(Tb/(1 + (0.00115* (Tb / 1.438))*log(Ep)))-273.15'
    lst = thermal.expression(lstExpression, {
        'Tb': thermal.select('B10'),
        'Ep': EM.select('EMM')
      }).rename('LST')

    # Get the timestamp of the image
    timestamp = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')

    # Get the mean LST value for the export region
    meanLST = lst.reduceRegion({
      'reducer': ee.Reducer.mean(),
      'geometry': geometry,
      'scale': 30,
      'maxPixels': 1e9
    }).get('LST')

    # Add the timestamp and mean LST value to the


List of all the images [{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7431, 7241], 'crs': 'EPSG:32643', 'crs_transform': [30, 0, 701085, 0, -30, 2667315]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7431, 7241], 'crs': 'EPSG:32643', 'crs_transform': [30, 0, 701085, 0, -30, 2667315]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7431, 7241], 'crs': 'EPSG:32643', 'crs_transform': [30, 0, 701085, 0, -30, 2667315]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7431, 7241], 'crs': 'EPSG:32643', 'crs_transform': [30, 0, 701085, 0, -30, 2667315]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7431, 7241], 'crs': 'EPSG:32643', 'crs_

EEException: ignored